## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Get the start time for computation.
start = time.time()

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yumen,CN,40.28,97.20,62.67,9,0,11.59,clear sky
1,1,Vaini,TO,-21.20,-175.20,73.56,100,20,3.44,few clouds
2,2,Zaltan,LY,32.95,11.87,69.10,40,0,15.48,clear sky
3,3,Busselton,AU,-33.65,115.33,59.97,79,99,12.39,overcast clouds
4,4,Okhotsk,RU,59.38,143.30,20.73,91,100,6.40,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

# Confirm the new dataframe was properly created.
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Pitimbu,BR,-7.47,-34.81,88.39,72,62,10.87,broken clouds
6,6,El Real De Santa Maria,PA,8.11,-77.73,76.33,97,91,1.92,overcast clouds
7,7,Rikitea,PF,-23.12,-134.97,79.16,72,0,16.73,clear sky
10,10,Keetmanshoop,NaN,-26.58,18.13,85.12,17,20,8.05,few clouds
14,14,Sabang,ID,5.89,95.32,84.07,69,58,7.18,broken clouds
...,...,...,...,...,...,...,...,...,...,...
670,670,Antsohihy,MG,-14.88,47.99,79.38,84,71,2.98,light rain
672,672,Campo Grande,BR,-20.44,-54.65,85.55,66,20,10.36,few clouds
677,677,Kulhudhuffushi,MV,6.62,73.07,84.56,68,34,4.27,scattered clouds
678,678,Astorga,BR,-23.23,-51.67,79.03,71,32,8.01,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.isnull().sum())
print(f"The dataframe has {len(preferred_cities_df)} entries.")

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64
The dataframe has 211 entries.


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(
    axis=0,
    how='any',
    inplace=True
)

# Confirm no more null values in dataframe.
print(preferred_cities_df.isnull().sum())
print(f"The dataframe has {len(preferred_cities_df)} entries.")

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64
The dataframe has 208 entries.


C:\Users\Daddy\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Pitimbu,BR,88.39,broken clouds,-7.47,-34.81,
6,El Real De Santa Maria,PA,76.33,overcast clouds,8.11,-77.73,
7,Rikitea,PF,79.16,clear sky,-23.12,-134.97,
14,Sabang,ID,84.07,broken clouds,5.89,95.32,
21,Axim,GH,86.00,overcast clouds,4.87,-2.24,
...,...,...,...,...,...,...,...
670,Antsohihy,MG,79.38,light rain,-14.88,47.99,
672,Campo Grande,BR,85.55,few clouds,-20.44,-54.65,
677,Kulhudhuffushi,MV,84.56,scattered clouds,6.62,73.07,
678,Astorga,BR,79.03,scattered clouds,-23.23,-51.67,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")

# Check the dataframe.
print(hotel_df.isnull().sum())
print(f"The dataframe has {len(hotel_df)} entries.")
print(hotel_df)

Reserva do Abiaí
Hotel not found... skipping.
People ThankYou
Hotel Citra
Axim Beach Resort And Spa
Hotel
Hotel Santika Luwuk
Piaçabuçu News
Hotel Apollo
Omadhoo Inn
Bils' Villa
Lorengau Harbourside Hotel
Finch Bay Galapagos Hotel
Chambres d'hôte "La Trinité"
Iftin hotel hobyo
Isles Sunset Lodge
Hôtel Le Dronmi
Bora-Bora Pearl Beach Resort & Spa
Sahid Bela Ternate Hotel
Pousada Lobo
Southern Sun Waterfront Cape Town
Paradise Inn
Hotel Santa Mônica
Aldea del Bazar Hotel and Spa
Leonardo Family Beach House 1
Villa Enata
Hotel Pekassa De Karche
محطة ام الارانب -سبها مواصلات
Marine Club Beach Resort
castelo pizzaria
Hotel not found... skipping.
CHACARA BAILLY
Hotel not found... skipping.
Blue Bay Lodge
HS Hotsson Smart Acapulco
أبو تركي للوحدات السكنيه
Hotel Midas Cornélio Procópio
Hôtel La Giralda
Auberge les abou
Fengshui Residence
Sur Plaza Hotel
The Fox Grove Inn
Nuevo Hotel Plaza
Hotel not found... skipping.
El Salvial
Hotel Xaguate
Conrad Bora Bora Nui
Taloa Heights
Pousada dos Naveg

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
print(hotel_df.isnull().sum())
print(f"The dataframe has {len(hotel_df)} entries.")
print(hotel_df)

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64
The dataframe has 190 entries.
               City Country  Max Temp Current Description    Lat     Lng  \
5           Pitimbu      BR     88.39       broken clouds  -7.47  -34.81   
7           Rikitea      PF     79.16           clear sky -23.12 -134.97   
14           Sabang      ID     84.07       broken clouds   5.89   95.32   
21             Axim      GH     86.00     overcast clouds   4.87   -2.24   
22        Mankachar      IN     82.65           clear sky  25.53   89.87   
..              ...     ...       ...                 ...    ...     ...   
670       Antsohihy      MG     79.38          light rain -14.88   47.99   
672    Campo Grande      BR     85.55          few clouds -20.44  -54.65   
677  Kulhudhuffushi      MV     84.56    scattered clouds   6.62   73.07   
678         Astorga  

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0),
                   zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations,
                                  info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Display total computational time.
end = time.time()
total_time = end - start
seconds = total_time % 3600
minutes = seconds // 60
seconds %= 60
print(f"Total time elapsed = {minutes:.0f} minutes {seconds:.0f} seconds")

Total time elapsed = 3 minutes 48 seconds
